# Generate Parallel Database

In [1]:
import torch 

def remove_by_index(db, index):
    return torch.cat((db[0:index], db[index+1:]),0)

def get_parallel_dbs(db):
    
    dbs = []
    for n in range(len(db)):
        parallel_db = remove_by_index(db, n)
        dbs.append(parallel_db)
    return dbs

def get_db_and_parallels(num_entries):
    db = torch.empty(num_entries).random_(2)
    pdbs = get_parallel_dbs(db)
    return db, pdbs

In [2]:
dbs = get_db_and_parallels(2000)

In [3]:
dbs

(tensor([0., 1., 1.,  ..., 1., 1., 0.]),
 [tensor([1., 1., 0.,  ..., 1., 1., 0.]),
  tensor([0., 1., 0.,  ..., 1., 1., 0.]),
  tensor([0., 1., 0.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1., 0.]),
  tensor([0., 1., 1.,  ..., 1., 1.,

# Towards Evaluating The Differential Privacy of Function


In [4]:
db, pdbs = get_db_and_parallels(20)

In [5]:
def query(db):
    return db.sum()

In [6]:
full_db_result = query(db)

In [7]:
# This is called the "L1 Sensitivity" or "Sensitivity" for short
# L1 because it's using absolute

"""
    SENSITIVTY: The maximum amount that the query 
                changes when removing an individual
                from the database.
                
                It allows us to understand how much
                a certain query leak or do not leak
                information.
"""
sensitivity = 0

for pdb in pdbs:
    pdb_result = query(pdb)
    
    # absolute becasuse we don't care if negative
    db_dinstance = torch.abs(full_db_result - pdb_result)
    
    if db_dinstance > sensitivity:
        sensitivity = db_dinstance
print(sensitivity)

tensor(1.)


In [8]:
# Consolodation all the code above to a single function

def get_sensitivity(query, n_entries):
    db, pdbs = get_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    sensitivity = 0

    for pdb in pdbs:
        pdb_result = query(pdb)

        # absolute becasuse we don't care if negative
        db_dinstance = torch.abs(full_db_result - pdb_result)

        if db_dinstance > sensitivity:
            sensitivity = db_dinstance
    return sensitivity

In [9]:
get_sensitivity(query, 100)

tensor(1.)

In [10]:
mean_query = lambda x: x.float().mean()

get_sensitivity(mean_query, 100)

"""
    Note that the purpose of the sensitivity function 
    is not to find how the function changes when a 
    value is removed, but about how much it changes
    when all the value that corespond to a certain 
    person are removed. Person, not individual value.
"""

'\n    Note that the purpose of the sensitivity function \n    is not to find how the function changes when a \n    value is removed, but about how much it changes\n    when all the value that corespond to a certain \n    person are removed. Person, not individual value.\n'

# Calculate L1 Sensitivity For Threshold

In [11]:
"""
    Steps:
        1. Create the query() function
        2. Create 10 databases of size 10
        3. Query each database with a 
           threshold of 5 (calculate
           sensitivity)
        4. Print out the sensitivity of 
           each database
"""

# Query function
def query(db, threshold=5):
    return (db.sum() > threshold).float()

# Step 2-3
for i in range(0,9):
    print(f"Sensitivity of DB {i+1}")
    print(get_sensitivity(query, 10))

Sensitivity of DB 1
0
Sensitivity of DB 2
0
Sensitivity of DB 3
0
Sensitivity of DB 4
0
Sensitivity of DB 5
tensor(1.)
Sensitivity of DB 6
0
Sensitivity of DB 7
0
Sensitivity of DB 8
0
Sensitivity of DB 9
0


So far, the sensitivity function that we've implmented is a **non-data coditioned sensitivity** function. That means that the sensitivity is based on the fucntion and what we know about the data. In this case, from a theoretical stand point, we know that maximum value or a maximum range of a sensitivity on this dataset is 1 (inclusing a threshold). Theoretically, this is what we want do during first time implementing differential privacy.

Note: I'm assuming that Andrew (the lecturer) means that we could now this by not actually looking at the database. 

However, if we take a peek at the data, we know that sometimes it's going to be one and sometimes it's not one. There are futher (more advance) sensitivity function, which is called **data conditioned sensitivity** that involves taking a peek at the dataset. So the sensitiviy can be calculated not just based on the range that we know that data could take, but also the actual values in the database.

# Perform a  Differencing on Row 10

Exlosing the value of a person represented by Row 10 in the database

Function to use:
- Sum
- Mean
- Threshodl

In [12]:
# Original DB
db, _= get_db_and_parallels(100)

# DB with Row 10 removed
pdb = remove_by_index(db, 10)

In [13]:
len(pdb), len(db)

(99, 100)

In [14]:
# Differencing attack via sum function

sum(db) - sum(pdb)

tensor(0.)

In [15]:
# Differencing attack via mean function

(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb) )

tensor(-0.0041)

In [16]:
# Differencing attack via threshold function

(sum(db).float() > 50).float() - (sum(pdb).float() > 50 ).float()

tensor(0.)

Differencing attack is near to the heart behind intuition of differencial privacy. Developing differencial privacy techniques involces the need of immunity against these kinds of attack.

# Simple Local Differential Privacy

Steps:
1. Create a coin flipper (1: heads, 0: tails)
2. Create database with size 10, 100, 1000, 10000
3. Modified for all database:
        for all value in database:
            flip1 -> flip coin
            flip2 -> flip coin
            if flip1 == 1, let value as it is
            otherwise, set value as flip2

In [ ]:
# Create a coin flipper (1: heads, 0: tails)
import random
def coin_flip():
    return random.choice([0,1])

# Create database with size 10, 100, 1000, 10000
ten, _ = get_db_and_parallels(10)
hundred, _ = get_db_and_parallels(100)
thousand, _ = get_db_and_parallels(1000)
ten_thousand, _ = get_db_and_parallels(10000)

# Function to add noise
def noise_db(db):
    new_db = []
    
    for i in range(len(db)):
        flip_1 = coin_flip()
        flip_2 = coin_flip()

        if flip_1 == 1:
            new_db.append(db[i])
        else:
            new_db.append(flip_2)
    return torch.FloatTensor(new_db)

# Cereate noised db
noised_ten = noise_db(ten)
noised_hundred = noise_db(hundred)
noised_thousand = noise_db(thousand)
noised_ten_thousand = noise_db(ten_thousand)

In [ ]:
def mean_query(db):
    return (db.sum() / len(db))

def print_db_noised_comparison(original_db, noised_db):
    original = mean_query(original_db)
    noised = mean_query(noised_db)
    output = f"DB of Size {len(original_db)}\nQuery of Original {original}\nQuery of Noised: {noised}\n"
    print(output)

In [ ]:
print_db_noised_comparison(ten, noised_ten)
print_db_noised_comparison(hundred, noised_hundred)
print_db_noised_comparison(thousand, noised_thousand)
print_db_noised_comparison(ten_thousand, noised_ten_thousand)

Above is my approach in solving the challenge. However, I'd suggest the following solution using mostly PyTorch by Andrew in the course.

In [17]:
db, _ = get_db_and_parallels(100)

flip_1 = (torch.rand(len(db)) > 0.5).float()
flip_2 = (torch.rand(len(db)) > 0.5).float()

augmented_db = db.float() * flip_1 + (1 - flip_1) * flip_2

Further explantion for the code above:

`db.float() * flip_1`: Here we want to get an array that contains which value will be the actual value and which one will be the value of the second flip of the coin. `flip_1` acts as a mask for the db values.

`(1 - flip_1)`: Contains the values where we want to put our random values of `flip_2`. All the 1s are the where we want the random values.

`(1 - flip_2) * flip_2`: All the values that are being sampled randomly.

Note: Based on the course, the man of the augmented database is 60%, which we can get by adding the "actual" statistic (in this case 70%) with 50% and diving the result by two. However, this might be different since our initialization of the database is random. In my case, the mean of the augmented database is 47%, and the "actual" statistic is 44%.

In [18]:
torch.mean(augmented_db.float()) 

tensor(0.5000)

In [19]:
db_result = torch.mean(augmented_db.float()) * 2 - 0.5
db_result

tensor(0.5000)

In [20]:
# Wrap it in to a single query function

def query(db):
    
    true_result = torch.mean(db.float())
    flip_1 = (torch.rand(len(db)) > 0.5).float()
    flip_2 = (torch.rand(len(db)) > 0.5).float()

    augmented_db = db.float() * flip_1 + (1 - flip_1) * flip_2
    db_result = torch.mean(augmented_db.float()) * 2 - 0.5
    
    return db_result, true_result

In [21]:
db, _ = get_db_and_parallels(10)
db_result, true_result = query(db)
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

With Noise: 0.10000002384185791
Without Noise: 0.4000000059604645


In [22]:
db, _ = get_db_and_parallels(100)
db_result, true_result = query(db)
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

With Noise: 0.42000001668930054
Without Noise: 0.49000000953674316


In [23]:
db, _ = get_db_and_parallels(1000)
db_result, true_result = query(db)
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

With Noise: 0.5080000162124634
Without Noise: 0.4749999940395355


In [24]:
db, _ = get_db_and_parallels(10000)
db_result, true_result = query(db)
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

With Noise: 0.5146000385284424
Without Noise: 0.49880000948905945


As we can see here, it seems tha the mean with and without the noise tend to be similar as the number of our dataste grows. This make sense since we are corupting the data points by adding noises. 

As local differential privacy is data hungry, the approach is suitable when we need to protect data at an individual level, with the condition that it is sufficiency large (not sure how large, but based on this implementaion it seems in the magnitude of tens of thousans suffice). 

On the other hand, we have global differential privacy which only add noise on the output, and this is suitable we have smaller dataset and still need to protect it.

# Varying Amount of Noise


In this challenge, we are supposed to add a parameter in the query function. The parameter will be the amount of noise that we want to apply to the first coin flip, meaning that there are going to be more 1s and 0s or vice versa.

The folowing is my solution before seeing the actual solution.

In [25]:
def query(db, flip_1_noise=0.5):
    
    true_result = torch.mean(db.float())
    flip_1 = (torch.rand(len(db)) > flip_1_noise).float()
    flip_2 = (torch.rand(len(db)) > 0.5).float()

    augmented_db = db.float() * flip_1 + (1 - flip_1) * flip_2
    db_result = torch.mean(augmented_db.float()) * 2 - 0.5
    
    return db_result, true_result

In [26]:
print("##### Noised with 0.7 probability for First Flip #####\n")

db, _ = get_db_and_parallels(10)
db_result, true_result = query(db, flip_1_noise=0.7)
print("DB of Size 10")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

db, _ = get_db_and_parallels(100)
db_result, true_result = query(db, flip_1_noise=0.7)
print("DB of Size 100")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

db, _ = get_db_and_parallels(1000)
db_result, true_result = query(db, flip_1_noise=0.7)
print("DB of Size 1000")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

db, _ = get_db_and_parallels(10000)
db_result, true_result = query(db, flip_1_noise=0.7)
print("DB of Size 10000")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

##### Noised with 0.7 probability for First Flip #####

DB of Size 10
With Noise: 0.7000000476837158
Without Noise: 0.5
DB of Size 100
With Noise: 0.5800000429153442
Without Noise: 0.47999998927116394
DB of Size 1000
With Noise: 0.5360000133514404
Without Noise: 0.4880000054836273
DB of Size 10000
With Noise: 0.49959999322891235
Without Noise: 0.5001999735832214


In [27]:
print("##### Noised with 0.2 probability for First Flip #####\n")

db, _ = get_db_and_parallels(10)
db_result, true_result = query(db, flip_1_noise=0.2)
print("DB of Size 10")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

db, _ = get_db_and_parallels(100)
db_result, true_result = query(db, flip_1_noise=0.2)
print("DB of Size 100")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

db, _ = get_db_and_parallels(1000)
db_result, true_result = query(db, flip_1_noise=0.2)
print("DB of Size 1000")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

db, _ = get_db_and_parallels(10000)
db_result, true_result = query(db, flip_1_noise=0.2)
print("DB of Size 10000")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

##### Noised with 0.2 probability for First Flip #####

DB of Size 10
With Noise: 0.5
Without Noise: 0.5
DB of Size 100
With Noise: 0.3999999761581421
Without Noise: 0.47999998927116394
DB of Size 1000
With Noise: 0.47600001096725464
Without Noise: 0.47699999809265137
DB of Size 10000
With Noise: 0.49140000343322754
Without Noise: 0.4973999857902527


Looks like everything runs well. It seems noe matter how much we add noise, the database with the nose and tihout seems to have similar means as the size of the dataset gets larger. 

**However**, there is a part that I actually missed after checking out the solution by Andrew. Specifically, it was this part

```
db_result = torch.mean(augmented_db.float()) * 2 - 0.5
```

How so? The thing is that in the previous implementation when the noise of the first flip is 0.5 is applied to the dataset; in other words, we want to apply the result of the second coin flip, where all the first coin flip values are 0s. By doing that, we know that 1/2 of the dataset's value are answered honestly, while the other 1/2 are all values of the second coin flip. 

based on the course, The "true" statistics was 70% (mine was 44%). This means that 1/2 of the dataset are 1s 70% of the time (equivalent t answering yes), while the other 1/2 has values of 1s 50% of the time. This is the case we know the "actual" statistic.

However, what we are doing here is pretending that we don't actually know the real statistic (the 70%). And we get is only the noised database mean, which was 60% based on the lecture. From that 60%, we know that one 1/2 of the dataset contained the true statistics, while the other one are 50/50 coin flips. Therefore, in order to get the actual statistic, we just need to do the following
```
0.6 * 2 - 0.5 = 70 
```

since 
```
(70 + 50) / 2 = 60 
```

The denoising method we used by averaging is sound since we know that the nosie of the dataset is 50% since it's a 50/50 coin flip. However, when we have a noise that is not necessarily 50/50, this method is not the best approach since the distribution has been applied a noise so that the mean is no longer around 0.5. Therefore, we need to use a differenr denoising method.

In the course, Andrew mentioned that we can see the averaging method above as the following
```
(true_db_mean * noise) + (noised_db_mean * (1 - noise))
```
If we insert the noise of 0.5 and do a little bit of algebra, then we actually have the same as averaging the true_db_mean and noised_db_mean by 2

```
noise = 0.5
(true_db_mean * noise) + (noised_db_mean * (1 - noise))
= 0.5 * true_db_mean  + 0.5 * noised_db_mean
= 1/2 * true_db_mean  + 1/2 * noised_db_mean
= (true_db_mean + noised_db_mean) / 2
```

And if we set `true_db_mean` as 70% and `noised_db_mean` as 50%, then we have
```
(0.7 * 0.5) + (0.5 * 0.5) = 0.6 = augmented_db_mean
```

Now we know that in order to get the mean of the augmented database, we can use the following
```
augmented_db_mean = (true_db_mean * noise) + (noised_db_mean * (1 - noise))
```

That means, with a little bit of algebra, we can find the formula to find the true mean of the database 
```
true_db_mean = (augmented_db_mean - (noised_db_mean * (1 - noise))) / noise
```

Knowing this, now we can modify the query function we created before.

In [28]:
def query(db, noise=0.5):
    
    true_result = torch.mean(db.float())
    flip_1 = (torch.rand(len(db)) > noise).float()
    flip_2 = (torch.rand(len(db)) > 0.5).float()

    augmented_db = db.float() * flip_1 + (1 - flip_1) * flip_2
    db_result = (augmented_db.float().mean() * (0.5 * (1 - noise))) / noise
    
    return db_result, true_result

In [29]:
db, _ = get_db_and_parallels(100)
db_result, true_result = query(db, noise=0.2)
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

With Noise: 0.6800000071525574
Without Noise: 0.3100000023841858


In [30]:
db, _ = get_db_and_parallels(100)
db_result, true_result = query(db, noise=0.4)
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

With Noise: 0.3974999785423279
Without Noise: 0.6000000238418579


In [31]:
db, _ = get_db_and_parallels(100)
db_result, true_result = query(db, noise=0.8)
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

With Noise: 0.06624999642372131
Without Noise: 0.4699999988079071


In [32]:
db, _ = get_db_and_parallels(10000)
db_result, true_result = query(db, noise=0.8)
print("DB of Size 10")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")

DB of Size 10
With Noise: 0.06212500482797623
Without Noise: 0.5029000043869019


In [ ]:
db, _ = get_db_and_parallels(10000)
db_result, true_result = query(db, noise=0.4)
print("DB of Size 10")
print(f"With Noise: {db_result}")
print(f"Without Noise: {true_result}")